In [1]:
import random
import simpy

In [2]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number de maquinas en la lavadora
WASHTIME = 5      # Minutos en limpiar un carro
T_INTER = 7       # Crea un coche cada ~ 7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos

In [133]:
class Carwash(object):
    #Un túnel de lavado tiene un número limitado de máquinas (` `NUM_MACHINES``) para
    #Limpiar coches en paralelo.
    #Los coches tienen que solicitar una de las máquinas. Cuando consiguieron uno, ellos
    #puede iniciar los procesos de lavado y esperar a que finalice (que
    #toma minutos de `` tiempo de lavado '').
    
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car, num_machines):
        #Los procesos de lavado. Se necesita un` `coche`` procesa y prueba para limpiarlo.
        
        yield self.env.timeout(WASHTIME)
        print("La lavadora %s elimino el %d%% de %s'." %
              (random.randint(1,2) ,random.randint(50, 99), car))
        
    def tiempo_salida(self, car):
        #Los procesos de lavado. Se necesita un` `coche`` procesa y prueba para limpiarlo.
        a = random.randint(2, 5)
        yield self.env.timeout(a)
        
    def tiempo_demora(self, car):
        #Los procesos de lavado. Se necesita un` `coche`` procesa y prueba para limpiarlo.
        a = random.randint(1, 5)
        yield self.env.timeout(a)

        
def car(env, name, cw, num_machines):
    #El proceso del auto (cada auto tiene un` `nombre '') llega al lavado de autos
    #(`` cw '') y solicita una máquina de limpieza.
    #Luego inicia el proceso de lavado, espera a que termine y
    #se va para no volver nunca ...
    print('El %s llega a la lavadora a las %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request
        
        yield env.process(cw.tiempo_demora(name))
        print("Tiempo demora del %s'  a las %.2f." % (name, env.now))

        print('El %s entra a la lavadora a las %.2f.' % (name, env.now))
        yield env.process(cw.wash(name,num_machines))

        print('El %s sale del tunel de lavado a las %.2f.' % (name, env.now))
        
        yield env.process(cw.tiempo_salida(name))
        print("Se libera la cola del %s' sale del tunel de lavado a las %.2f." % (name, env.now))
        


def setup(env, num_machines, washtime, t_inter):
    #Crea un lavado de autos, varios autos iniciales y sigue creando autos
    #aprox. cada `` t_inter`` minutos.
    # Creamos la lavadora
    #num_machines = random.randrange(1,3)
    carwash = Carwash(env, num_machines, washtime)

    # Creamos los carros iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash,num_machines))

    # Crea más coches mientras se ejecuta la simulación
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash,num_machines))

In [134]:
# Configurar e iniciar la simulación

random.seed(RANDOM_SEED)  
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

<Process(setup) object at 0x7f9ea17c5c40>

In [135]:
print('Lavadora UPS')
env.run(until=SIM_TIME)

Lavadora UPS
El Carro 0 llega a la lavadora a las 0.00.
El Carro 1 llega a la lavadora a las 0.00.
El Carro 2 llega a la lavadora a las 0.00.
El Carro 3 llega a la lavadora a las 0.00.
Tiempo demora del Carro 0'  a las 1.00.
El Carro 0 entra a la lavadora a las 1.00.
Tiempo demora del Carro 1'  a las 3.00.
El Carro 1 entra a la lavadora a las 3.00.
El Carro 4 llega a la lavadora a las 5.00.
La lavadora 1 elimino el 58% de Carro 0'.
El Carro 0 sale del tunel de lavado a las 6.00.
La lavadora 1 elimino el 87% de Carro 1'.
El Carro 1 sale del tunel de lavado a las 8.00.
Se libera la cola del Carro 0' sale del tunel de lavado a las 8.00.
Tiempo demora del Carro 2'  a las 9.00.
El Carro 2 entra a la lavadora a las 9.00.
El Carro 5 llega a la lavadora a las 11.00.
Se libera la cola del Carro 1' sale del tunel de lavado a las 13.00.
La lavadora 1 elimino el 64% de Carro 2'.
El Carro 2 sale del tunel de lavado a las 14.00.
Tiempo demora del Carro 3'  a las 14.00.
El Carro 3 entra a la lavadora

2
